# Sequential methods to build e neural net

In [31]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [32]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [33]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)
        
    def __getitem__(self, n):
        return self.x[n], self.y[n]
    
    def __len__(self):
        return len(self.x)

In [34]:
ds = MyDataset(x, y)
dl = DataLoader(ds, batch_size=2, shuffle=True)

model = nn.Sequential(
        nn.Linear(2, 8),
        nn.ReLU(),
        nn.Linear(8, 1)
    ).to(device)


In [35]:
#summary(model, torch.zeros(1, 2))

loss_func = nn.MSELoss()

from torch.optim import SGD
opt = SGD(model.parameters(), lr = 0.001)

import time
loss_hisotry = []

start = time.time()

for _ in range(50):
    for ix, iy in dl:
        opt.zero_grad()
        loss_value = loss_func(model(ix), iy)
        loss_value.backward()
        opt.step()
        loss_hisotry.append(loss_value)

end = time.time()

print(end-start)

0.12926292419433594


In [36]:
val = [[8,9], [10,11], [11,12], [1.5,2.5]]
val_T = torch.tensor(val).float().to(device)
model(val_T)

tensor([[16.5960],
        [20.2760],
        [22.1160],
        [ 4.4492]], device='mps:0', grad_fn=<LinearBackward0>)

In [37]:
torch.save(model.to('cpu').state_dict(), 'nntest.pth')

In [38]:
saved_model_state = torch.load('nntest.pth')

saved_model = nn.Sequential(
        nn.Linear(2, 8),
        nn.ReLU(),
        nn.Linear(8, 1)
    )

saved_model.load_state_dict(saved_model_state)
saved_model.to(device)

saved_model(val_T)

tensor([[16.5960],
        [20.2760],
        [22.1160],
        [ 4.4492]], device='mps:0', grad_fn=<LinearBackward0>)